In [ ]:
import numpy as np
from mathphys.functions import load
import pyaccel
from pymodels import si
import matplotlib.pyplot as plt
import os
from apsuite.loco import LOCOAnalysis
from apsuite.orbcorr import OrbRespmat

In [ ]:
def get_all_loco_fitting_folders(folder, prefix=''):
    folder += '' if folder.endswith('/') else '/'
    return sorted([
        folder + f for f in os.listdir(prefix + folder) if '.' not in f and 'IVU_EMA_ACORM' in f])

def get_idcs(fam_name, mod, idc_delta):
    idc = np.array(pyaccel.lattice.find_indices(mod, 'fam_name', fam_name))
    idx = np.argsort(np.abs(idc_delta-idc))[:2]
    return idc[idx]

# Load Data

In [ ]:
allfols = []
allfols += get_all_loco_fitting_folders('./')

kparams = np.array([float(fname.split('/')[-1].split('_')[-1].replace('p', '.')) for fname in allfols])
polarizations = np.array([fname.split('/')[-1].split('_')[-3] for fname in allfols])
runs = np.array([fname.split('/')[0] for fname in allfols])

In [ ]:
# Construct loco analysis
alllocoanls = []
fname_fittings_ = []
for fol in allfols:
    parent, fol = os.path.split(fol)
    fname_input = f'{parent}/loco_input_{fol}.pickle'
    fname_fit = f'{parent}/{fol}/fitting_{fol}.pickle'
    fname_fittings_.append(fname_fit)
    anl = LOCOAnalysis(
        fname_setup= fname_input,
        fname_fit= fname_fit,
    )
    anl.get_setup()
    alllocoanls.append(anl)
fname_fittings = np.array(fname_fittings_).ravel()

In [ ]:
ref_model = load('./IVU_EMA_ACORM_kparam_24p000/fitting_IVU_EMA_ACORM_kparam_24p000.pickle')['fit_model']
fam = si.get_family_data(ref_model)

idc_delta = pyaccel.lattice.find_indices(ref_model, 'fam_name', 'IVU18')[0]
idc_qfb = []
idc_qdb1 = []
idc_qdb2 = []
idc_qfb.append(get_idcs('QFB', ref_model, idc_delta))
idc_qdb1.append(get_idcs('QDB1', ref_model, idc_delta))
idc_qdb2.append(get_idcs('QDB2', ref_model, idc_delta))

idcs_qfb = np.array(idc_qfb).ravel()
idcs_qdb1 = np.array(idc_qdb1).ravel()
idcs_qdb2 = np.array(idc_qdb2).ravel()

idcs_qn = np.array(fam['QN']['index']).ravel()
kl0 = pyaccel.lattice.get_attribute(ref_model, 'KL', idcs_qn)


kl0_qfb = pyaccel.lattice.get_attribute(ref_model, 'KL', idcs_qfb)
kl0_qdb1 = pyaccel.lattice.get_attribute(ref_model, 'KL', idcs_qdb1)
kl0_qdb2 = pyaccel.lattice.get_attribute(ref_model, 'KL', idcs_qdb2)

In [ ]:
klf = np.zeros((len(idcs_qn), len(fname_fittings)))
klf_qfb = np.zeros((len(idcs_qfb), len(fname_fittings)))
klf_qdb1 = np.zeros((len(idcs_qdb1), len(fname_fittings)))
klf_qdb2 = np.zeros((len(idcs_qdb2), len(fname_fittings)))

for i, fname in enumerate(fname_fittings):
    fitmodel = load(fname)['fit_model']
    klf[:, i] = pyaccel.lattice.get_attribute(fitmodel, 'KL', idcs_qn)
    klf_qfb[:, i] = pyaccel.lattice.get_attribute(fitmodel, 'KL', idcs_qfb)
    klf_qdb1[:, i] = pyaccel.lattice.get_attribute(fitmodel, 'KL', idcs_qdb1)
    klf_qdb2[:, i] = pyaccel.lattice.get_attribute(fitmodel, 'KL', idcs_qdb2)

# Check KL variations

In [ ]:
plt.figure()
plt.title('QFB')
plt.plot(kparams, 1e3*(klf_qfb[0, :]-kl0_qfb[0]), label='upstream')
plt.plot(kparams, 1e3*(klf_qfb[1, :]-kl0_qfb[1]), label='downstream')
plt.grid()
plt.legend()
plt.xlabel('kparam [mm]')
plt.ylabel('K [1/km]')
plt.show()

In [ ]:
plt.figure()
plt.title('QDB1')
plt.plot(kparams, 1e3*(klf_qdb1[0, :]-kl0_qdb1[0]), label='upstream')
plt.plot(kparams, 1e3*(klf_qdb1[1, :]-kl0_qdb1[1]), label='downstream')
plt.grid()
plt.legend()
plt.xlabel('kparam [mm]')
plt.ylabel('K [1/km]')
plt.show()

In [ ]:
plt.figure()
plt.title('QDB2')
plt.plot(kparams, 1e3*(klf_qdb2[0, :]-kl0_qdb2[0]), label='upstream')
plt.plot(kparams, 1e3*(klf_qdb2[1, :]-kl0_qdb2[1]), label='downstream')
plt.grid()
plt.legend()
plt.xlabel('kparam [mm]')
plt.ylabel('K [1/km]')
plt.show()

# Check LOCO tunes and ORMs

In [ ]:
respm_meas0 = load('loco_input_IVU_EMA_ACORM_kparam_24p000.pickle')['respmat']

orb_respm0 = OrbRespmat(ref_model, 'SI')
respmat0 = orb_respm0.get_respm()
respmat0[:, -1] *= 1e6

respmats = np.zeros((respmat0.shape[0], respmat0.shape[1], len(fname_fittings)))
dmatf = np.zeros((np.product(respm_meas0.shape), len(fname_fittings)))
for i, fname in enumerate(fname_fittings):
    fitmodel = load(fname)['fit_model']
    orb_respmf = OrbRespmat(fitmodel, 'SI')
    respmatf = orb_respmf.get_respm()
    respmatf[:, -1] *= 1e6
    respmats[:, :, i] = respmatf
    dmatf[:, i] = (respmats[:, :, i] - respm_meas0).ravel()
std = dmatf.std(axis=0)
plt.plot(std)

In [ ]:
twiss0, *_ = pyaccel.optics.calc_twiss(ref_model, indices='open')
mux0 = twiss0.mux[-1]/2/np.pi
muy0 = twiss0.muy[-1]/2/np.pi
mux = np.zeros(len(fname_fittings))
muy = np.zeros(len(fname_fittings))
for i, fname in enumerate(fname_fittings):
    fitmodel = load(fname)['fit_model']
    twiss, *_ = pyaccel.optics.calc_twiss(fitmodel, indices='open')
    mux[i] = twiss.mux[-1]/2/np.pi - mux0
    muy[i] = twiss.muy[-1]/2/np.pi - muy0
plt.plot(kparams, muy)